In [33]:
from filter_datasets import *
import json
import ppx #A Python interface to proteomics data repositories
from pyteomics import mztab
from ete3 import Tree, NodeStyle, TreeStyle, NodeStyle, faces, AttrFace, CircleFace
from ete3 import NCBITaxa

## 1. Get MassIVE

In [34]:
databases_massIVE = getAllDts() #get all datesets available on massIVE
MassIVE_datasets_df = pd.DataFrame(databases_massIVE)
MassIVE_datasets_df.species[MassIVE_datasets_df.species.str.contains('3702')] # Arabidopsis positive control. Search made through taxon ID
MassIVE_datasets_df.head()

,dataset,datasetNum,title,user,site,flowname,createdMillis,created,description,fileCount,...,keywords,pi,complete,quant_analysis,status,private,hash,px,task,id
0,MSV000088303,88303,GNPS - DDA analysis on leaf extracts of 6 Aste...,kbkang,MassIVE,MASSIVE-COMPLETE,1635921029000,"Nov. 2, 2021, 11:30 PM",LC-Q/TOF DDA analysis on leaf extracts of 6 As...,10,...,Aster;leaf,"[{'name': 'Kyo Bin Kang ', 'email': 'kbkang@so...",false,Quantification Results,Partial,false,,,bb6a7269a69749f897998354219db189,0
1,MSV000088302,88302,Increasing the throughput of sensitive proteom...,jderks,MassIVE,MASSIVE-COMPLETE,1635891242000,"Nov. 2, 2021, 3:14 PM",Mass-spectrometry methods enable high-throughp...,233,...,plexDIA;DIA;mTRAQ;Cell cycle;high-throughput s...,"[{'name': 'Nikolai Slavov', 'email': 'n.slavov...",false,Quantification Results,Partial,false,,PXD029531,8b0a2f5b2fc84964b4bd4ee64fc84d25,1
2,MSV000088298,88298,The landscape of the SARS-CoV-2 HLA-peptidome,ccms,MassIVE,MASSIVE-COMPLETE,1635786592000,"Nov. 1, 2021, 10:09 AM",The human leukocyte antigen (HLA)-bound-viral ...,259,...,Sars-cov-2; Immunopeptidomics; Hla-peptidomics,"[{'name': 'Prof. Yardena Samuels', 'email': 'y...",false,null,Partial,false,,PXD023614,9c26cd591151455b854e97d9f0c6cfe0,2
3,MSV000088291,88291,B metadata-massive datasets upload-2021-11-1,xuyaping_1995,MassIVE,MASSIVE-COMPLETE,1635738512000,"Oct. 31, 2021, 8:48 PM",Screening and identifying PAs of chinese medic...,10,...,"Chinese medicines, Pyrrolizidine alkaloids, no...","[{'name': 'Sunshine', 'email': 'sunshine_xyp@1...",false,null,Partial,false,,,f6028f8d889a4ce3b6cc625b8f9ddeac,3
4,MSV000088290,88290,TIMSFolding allows the use of ion mobility enh...,orsburn,MassIVE,MASSIVE-COMPLETE,1635699235000,"Oct. 31, 2021, 9:53 AM",Files associated with the TIMSFolding preprint...,41,...,TIMSTOF;Spectral libraries;DDA;Historic softwa...,"[{'name': 'Benjamin Orsburn', 'email': 'borsbu...",false,null,Partial,false,,PXD029447,f742e08a9f634f4c9fb16261bbc25b8c,4


In [35]:
# massive_plantid_proteomics.tsv is the result from filterMassIVE function on MassIVE databases
MassIVE_passed_df = pd.read_csv('massive_plantid_proteomics.tsv', sep='\t')
MassIVE_passed_df.head()

,dataset,datasetNum,title,user,site,flowname,createdMillis,created,description,fileCount,...,pi,complete,quant_analysis,status,private,hash,px,task,id,search
0,MSV000087822,87822,TMK-based cell surface auxin signaling activat...,haiyanzheng,MassIVE,MASSIVE-COMPLETE,1626290200000,"Jul. 14, 2021, 12:16 PM",The phytohormone auxin controls a myriad of pr...,72,...,"[{'name': 'Yang, zhenbiao', 'email': 'zhenbiao...",False,NaN,Partial,False,NaN,NaN,ab62b7b1d5e5455dac665ed8629caf97,119,0
1,MSV000087472,87472,Complement of proteins associated with clathri...,dahhan,MassIVE,MASSIVE-COMPLETE,1621570970000,"May. 20, 2021, 9:22 PM","In Eukaryotes, clathrin coated vesicles (CCVs)...",43,...,"[{'name': 'Sebastian Y. Bednarek', 'email': 's...",True,NaN,Complete,False,NaN,PXD026180,d36fa49fcec846a288df2ffd04669068,260,1
2,MSV000086794,86794,Changes in mitochondrial proteome upon RNAi kn...,minsookim,MassIVE,MASSIVE-COMPLETE,1612266144000,"Feb. 2, 2021, 3:42 AM",To investigate how reduced ATP synthase subuni...,21,...,"[{'name': 'Elizabeth Vierling', 'email': 'vier...",False,NaN,Partial,False,NaN,NaN,b9b6448f3c534d37ae178b646f45ac3b,519,1
3,MSV000086324,86324,NOT9 Co-IP in Arabidopsis thaliana,klement,MassIVE,MASSIVE-COMPLETE,1603120368000,"Oct. 19, 2020, 8:12 AM",Interacting partners of NOT9 in Arabidopsis th...,13,...,"[{'name': 'Eva Klement', 'email': 'klement.eva...",False,NaN,Partial,False,NaN,NaN,910cf186e64d4d8593e9d5eecefc6c18,732,0
4,MSV000086248,86248,SA-Induced MTD-FLAG Interactome in A. thaliana,tch21,MassIVE,MASSIVE-COMPLETE,1602159199000,"Oct. 8, 2020, 5:13 AM",Identification of the salicylic acid (SA) indu...,506,...,"[{'name': 'John Williamson', 'email': 'jdw@ncs...",True,NaN,Complete,False,NaN,PXD021892,4b0caa7982ec4bbf9bc3c7c68ad1d6a2,760,0


In [36]:
MassIVE_passed_df.columns

Index(['dataset', 'datasetNum', 'title', 'user', 'site', 'flowname',
       'createdMillis', 'created', 'description', 'fileCount', 'fileSizeKB',
       'spectra', 'psms', 'peptides', 'variants', 'proteins', 'search_psms',
       'search_peptides', 'search_variants', 'search_proteins',
       'search_spectra', 'reanalysis_psms', 'reanalysis_peptides',
       'reanalysis_variants', 'reanalysis_proteins', 'species', 'instrument',
       'modification', 'keywords', 'pi', 'complete', 'quant_analysis',
       'status', 'private', 'hash', 'px', 'task', 'id', 'search'],
      dtype='object')

In [43]:
MassIVE_passed_proteins_df = MassIVE_passed_df[MassIVE_passed_df.proteins > 0]
MassIVE_passed_proteins_df.head()

,dataset,datasetNum,title,user,site,flowname,createdMillis,created,description,fileCount,...,pi,complete,quant_analysis,status,private,hash,px,task,id,search
1,MSV000087472,87472,Complement of proteins associated with clathri...,dahhan,MassIVE,MASSIVE-COMPLETE,1621570970000,"May. 20, 2021, 9:22 PM","In Eukaryotes, clathrin coated vesicles (CCVs)...",43,...,"[{'name': 'Sebastian Y. Bednarek', 'email': 's...",True,NaN,Complete,False,NaN,PXD026180,d36fa49fcec846a288df2ffd04669068,260,1
4,MSV000086248,86248,SA-Induced MTD-FLAG Interactome in A. thaliana,tch21,MassIVE,MASSIVE-COMPLETE,1602159199000,"Oct. 8, 2020, 5:13 AM",Identification of the salicylic acid (SA) indu...,506,...,"[{'name': 'John Williamson', 'email': 'jdw@ncs...",True,NaN,Complete,False,NaN,PXD021892,4b0caa7982ec4bbf9bc3c7c68ad1d6a2,760,0
7,MSV000085233,85233,Structure elucidation for tunable hetero-assem...,alchemistmatt,MassIVE,MASSIVE-COMPLETE,1586186955000,"Apr. 6, 2020, 8:29 AM",Mass spectrometry raw data including peptide m...,60,...,"[{'name': 'Mowei Zhou', 'email': 'mowei.zhou@p...",True,Quantification Results,Complete,False,NaN,NaN,7596321a25b64573bb4759a416bd24b1,1309,1
17,MSV000082175,82175,Arabidopsis circadian proteomics and phosphopr...,JKrahmer,MassIVE,MASSIVE-COMPLETE,1521208967000,"Mar. 16, 2018, 7:02 AM",Proteomics and phosphoproteomics analyses of c...,1445,...,"[{'name': 'Prof Andrew Millar', 'email': 'Andr...",True,NaN,Complete,False,NaN,PXD009230,2f7f4ada5f7b4aceaaf5971de10e4eb8,3118,0
20,MSV000081715,81715,Dynamic Acclimation to High Light in Arabidops...,ccms,MassIVE,MASSIVE-COMPLETE,1510698521000,"Nov. 14, 2017, 2:28 PM","Typically, when fully developed leaves of Arab...",142,...,"[{'name': 'Giles N Johnson', 'email': 'giles.j...",True,NaN,Complete,False,NaN,PXD006598,c717cb7c5fe5497d92c7b471da159577,3355,0


In [66]:
MassIVE_obj = ppx.MassiveProject("MSV000087472", local = "/home/tiago/documents/lncRNA/massive")
MassIVE_remote_files = MassIVE_obj.remote_files()
MassIVE_remote_files = list(filter(lambda x: (x.endswith(".mzTab")),MassIVE_remote_files))
MassIVE_remote_files

['ccms_result/Dataset A_Replicate 1_Mudpit_ft2010122202 (F020529).mzTab',
 'ccms_result/Dataset A_Replicate 2_Mudpit_lt2011111105 (F020530).mzTab',
 'ccms_result/Dataset A_Replicate 3_Mudpit_qe2013101702 (F020531).mzTab',
 'ccms_result/Dataset A_Replicate 4_Mudpit_qe2013121702 (F020548).mzTab']

In [67]:
MassIVE_mztab_download = MassIVE_obj.download("ccms_result/Dataset A_Replicate 1_Mudpit_ft2010122202 (F020529).mzTab")

TOTAL: 100%|██████████| 1/1 [00:01<00:00,  1.07s/files]


In [68]:
MassIVE_mztab = mztab.MzTab("/home/tiago/documents/lncRNA/massive/ccms_result/Dataset A_Replicate 1_Mudpit_ft2010122202 (F020529).mzTab")
psms = MassIVE_mztab.spectrum_match_table
psms.head()

,sequence,PSM_ID,accession,unique,database,database_version,search_engine,search_engine_score[1],search_engine_score[2],search_engine_score[3],...,post,start,end,opt_global_mzidentml_original_ID,opt_global_cv_MS:1002217_decoy_peptide,opt_global_cv_PRIDE:0000091_rank,opt_global_pass_threshold,opt_global_valid,opt_global_invalid_reason,opt_global_cv_MS:1002354_PSM-level_q-value
PSM_ID,,,,,,,,,,,,,,,,,,,,,
1,CLKSCFVGMR,1,AT1G22110.1,None,TAIR10_pep_20101214_FR_CP_BSA.fasta,Unknown,"[Scaffold, Mascot]",0.151114,0.12,32.407986,...,None,264,273,Spec_50587_CLKSC+57FVGM+16R,0,1,false,VALID,None,None
2,FAMPLSR,2,AT1G12410.1,None,TAIR10_pep_20101214_FR_CP_BSA.fasta,Unknown,"[Scaffold, Mascot]",0.805878,15.00,34.739246,...,None,183,189,Spec_73945_FAMPLSR,0,1,false,VALID,None,None
3,LTIPR,3,AT1G34220.2,None,TAIR10_pep_20101214_FR_CP_BSA.fasta,Unknown,"[Scaffold, Mascot]",0.100000,5.07,25.276299,...,None,23,27,Spec_56822_LTIPR,0,1,false,VALID,None,None
4,YDQGSSIRKEK,4,AT1G56030.1,None,TAIR10_pep_20101214_FR_CP_BSA.fasta,Unknown,"[Scaffold, Mascot]",0.156273,1.00,34.777000,...,None,265,275,Spec_60321_YDQGSSIRKEK,0,1,false,VALID,None,None
5,NLLETCFTGQKNLK,5,AT1G56030.1,None,TAIR10_pep_20101214_FR_CP_BSA.fasta,Unknown,"[Scaffold, Mascot]",0.159623,7.36,35.317340,...,None,242,255,Spec_76896_NLLETCFTGQKNLK,0,1,false,VALID,None,None


In [72]:
unknown = MassIVE_mztab["PRT"].description.str.contains('unknown').sum()
putative = MassIVE_mztab["PRT"].description.str.contains('putative').sum()
hypothetical = MassIVE_mztab["PRT"].description.str.contains('hypothetical').sum()
RNA = MassIVE_mztab["PRT"].description.str.contains('RNA').sum()
coding = MassIVE_mztab["PRT"].description.str.contains('coding').sum()
print(f"Words used as baits:\n Unknown: {unknown}\n putative: {putative} \n hypothetical: {hypothetical} \n RNA: {RNA} \n Coding: {coding}")


Words used as baits:
 Unknown: 1089
 putative: 80 
 hypothetical: 4 
 RNA: 314 
 Coding: 0


In [ ]:
#Tried to find zmtable on massIVE datasets. 
for dataset in MassIVE_passed_df["dataset"]:  
    massive_obj = ppx.MassiveProject(
        f"{dataset}")
    massive_remote_files = massive_obj.remote_files(glob = ".mztable")
    if massive_remote_files:
        print(massive_remote_files)
    

## 2. GET PRIDE

In [ ]:
# pride_plant.json is the result from filterPride function on PRIDE databases
with open('pride_plant.json') as pride_plants_file:
    pride_plant = json.load(pride_plants_file)
print(f"Total of plants retrieved from pride database: {len(pride_plant)}")

In [ ]:
projetc_identifiers = [] # holds project number for all plants species data from pride
accession_number = [] # holds taxon id for all plants species data from pride
for key,value in pride_plant.items():
    projetc_identifiers.append(key)
    accession_number.append(value['accession'])
    

In [ ]:
projetc_identifiers[:5]

In [ ]:
accession_number[:5]

In [ ]:
# pride changed its FPT link, so the ppx brokes
proj = ppx.find_project(projetc_identifiers[0])
proj_remote_files = proj.remote_files()


In [ ]:
# Try to accesss files through link available on metadata file
with open('pride.log.json',"w+") as pride_log_file:
    for all_projects in projetc_identifiers[:10]:
        proj = ppx.find_project(all_projects)
        print(proj.id)
        print(proj.metadata["_links"]["files"]["href"])
        json.dump(proj.metadata,pride_log_file) # generates a log files in case manual checks in necessary. Takes some time


# remote_files = proj.remote_files()
# print(remote_files)

In [ ]:
findmztab(projetc_identifiers[0])==None # select m/z tables from a given project.  

In [ ]:
#Search for project with mztab in pride plants database
for i in projetc_identifiers:
    if findmztab(i)!= None: #Selec only those files with m\z tables
        print(i)
        break #stops at fist result 

In [ ]:
#Explores the first result
#Everything from here must be done to all findings 
proj = ppx.find_project('PRD000084')

In [ ]:
proj.metadata[""]


In [ ]:
pride_plant['PRD000084']

In [ ]:
remote_files = proj.remote_files()
print(remote_files)

In [ ]:
findmztab(i)

In [ ]:
downloaded = proj.download("generated/PRIDE_Exp_Complete_Ac_15505.pride.mztab.gz")
print(downloaded)

In [ ]:
!gzip -df /home/tiago/.ppx/PRD000084/generated/PRIDE_Exp_Complete_Ac_15505.pride.mztab.gz 

In [ ]:
!ls /home/tiago/.ppx/PRD000084/generated/

In [ ]:
tables = mztab.MzTab("/home/tiago/.ppx/PRD000084/generated/PRIDE_Exp_Complete_Ac_15505.pride.mztab")
psms = tables.spectrum_match_table
psms.head()

In [ ]:
tables.keys()

In [ ]:
prt = pd.DataFrame(tables['PRT'])

In [ ]:
prt.head()

In [ ]:
prt['description'].str.contains('unknown').sum() #Number of unknown proteins confirmed 

In [ ]:
prt["description"].str.contains("putative").sum() #Number of putative proteins 

In [ ]:
prt["description"].str.contains("hypothetical").sum() #Nuber of hypothetical proteins

In [ ]:
prt['description'].str.contains('lnc').sum() #Number of long non coding RNA?

In [ ]:
prt['description'].str.contains('coding', case=False).sum() #Number of coding RNA?

In [ ]:
prt['description'].str.contains('non', case=False).sum() #Number of coding RNA

In [ ]:
idx = prt['description'].str.contains('unknown') #see the proteins found
idx

In [ ]:
idx[0] = False
prt[idx]

In [ ]:
accession = list(set(accession))

print(len(accession))
accession[:10]

In [ ]:
ncbi = NCBITaxa()


tree = ncbi.get_topology(accession[:3])

print (tree.get_ascii(attributes=["sci_name", "rank"]))

# Not working, make plot of species available, can use other tools
# https://phylot.biobyte.de
ts = TreeStyle()
ts.show_leaf_name = True
ts.mode = "c"
ts.arc_start = -180 # 0 degrees = 3 o'clock
ts.arc_span = 180

for n in tree.traverse():
    if n.is_leaf():
        n.name = n.sci_name

tree.render("pride_plant_tree.pdf", w=500, tree_style=ts)